In [1]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
%config Completer.use_jedi = False

In [2]:
torch.manual_seed(0)

x_data = torch.randint(1, 9, (2, 3))
print(x_data)

emb = nn.Embedding(9, 3)

x_emb = emb(x_data)

print("x_emb")
print(x_emb)

rnn = nn.RNN(3, 5, batch_first = True, bidirectional=True, bias = False)

for name, param in rnn.named_parameters():
    print(name, param.shape)
    
# print(rnn.weight_hh_l0)
# print()
# print(rnn.weight_hh_l0_reverse)

out, hidden = rnn(x_emb)
print("Pytorch results:")
print("out:")
print(out)
print()
print("hidden:")
print(hidden)

tensor([[5, 8, 6],
        [1, 4, 4]])
x_emb
tensor([[[ 0.1124,  0.6408,  0.4412],
         [-1.4689, -1.5867,  1.2032],
         [-0.2159,  0.7924, -0.2897]],

        [[ 0.4372,  0.4913, -0.2041],
         [-0.4927,  0.2484,  0.4397],
         [-0.4927,  0.2484,  0.4397]]], grad_fn=<EmbeddingBackward>)
weight_ih_l0 torch.Size([5, 3])
weight_hh_l0 torch.Size([5, 5])
weight_ih_l0_reverse torch.Size([5, 3])
weight_hh_l0_reverse torch.Size([5, 5])
Pytorch results:
out:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897, -0.2995, -0.2097,
           0.5120, -0.3699, -0.2247],
         [ 0.1449,  0.0654,  0.0732,  0.6874,  0.3344,  0.3512, -0.8475,
           0.1094,  0.9039, -0.3021],
         [ 0.2900,  0.1927,  0.0584, -0.1282, -0.3534,  0.1210,  0.1893,
           0.2093, -0.2455, -0.1848]],

        [[ 0.0427,  0.0461, -0.0034, -0.2824, -0.0789, -0.2437,  0.2922,
           0.0279, -0.4758,  0.1251],
         [ 0.2271, -0.0165,  0.1382,  0.0907, -0.0504, -0.0109, -0.3373,
         

In [3]:
my_x = x_emb.permute(1, 0, 2)
print(my_x)

my_x_reverse = torch.flip(my_x, [0, 1])
print(my_x_reverse)

tanh = nn.Tanh()

wii = rnn.weight_ih_l0
whh = rnn.weight_hh_l0

print("Calculating the results from RNN in forward direction")
batch_size, seq_len, feat_len = x_emb.shape

hiddens = []
h_t = torch.zeros(batch_size, 5)
for i in range(seq_len):
    ii = my_x[i] @ torch.transpose(wii, 0, 1)
    hh = h_t @ torch.transpose(whh, 0, 1)
    
    h_t = tanh(ii + hh)
    
    hiddens.append(h_t)

my_out_forward = torch.stack(hiddens).permute(1, 0, 2)

print()
print("My out forward:")
print(my_out_forward)

tensor([[[ 0.1124,  0.6408,  0.4412],
         [ 0.4372,  0.4913, -0.2041]],

        [[-1.4689, -1.5867,  1.2032],
         [-0.4927,  0.2484,  0.4397]],

        [[-0.2159,  0.7924, -0.2897],
         [-0.4927,  0.2484,  0.4397]]], grad_fn=<PermuteBackward>)
tensor([[[-0.4927,  0.2484,  0.4397],
         [-0.2159,  0.7924, -0.2897]],

        [[-0.4927,  0.2484,  0.4397],
         [-1.4689, -1.5867,  1.2032]],

        [[ 0.4372,  0.4913, -0.2041],
         [ 0.1124,  0.6408,  0.4412]]], grad_fn=<FlipBackward>)
Calculating the results from RNN in forward direction

My out forward:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897],
         [ 0.1449,  0.0654,  0.0732,  0.6874,  0.3344],
         [ 0.2900,  0.1927,  0.0584, -0.1282, -0.3534]],

        [[ 0.0427,  0.0461, -0.0034, -0.2824, -0.0789],
         [ 0.2271, -0.0165,  0.1382,  0.0907, -0.0504],
         [ 0.3176,  0.1364,  0.0548,  0.2315, -0.1117]]],
       grad_fn=<PermuteBackward>)


In [5]:
wii_r = rnn.weight_ih_l0_reverse
whh_r = rnn.weight_hh_l0_reverse

print("Calculating the results from RNN in backward direction")
# batch_size, seq_len, feat_len = x_emb.shape

hiddens = []
h_t = torch.zeros(batch_size, 5)

for i in range(seq_len):
    ii = my_x_reverse[i] @ torch.transpose(wii_r, 0, 1)
    hh = h_t @ torch.transpose(whh_r, 0, 1)
    
    h_t = tanh(ii + hh)
    
    hiddens.append(h_t)

my_out_backward = torch.flip(torch.stack(hiddens).permute(1, 0, 2),[0, 1])

# print()
# print("My hiddens backward:")
# print(hiddens)

print()
print("My out forward:")
print(my_out_forward)
print()
print("My out bacward:")
print(my_out_backward)

Calculating the results from RNN in backward direction

My out forward:
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897],
         [ 0.1449,  0.0654,  0.0732,  0.6874,  0.3344],
         [ 0.2900,  0.1927,  0.0584, -0.1282, -0.3534]],

        [[ 0.0427,  0.0461, -0.0034, -0.2824, -0.0789],
         [ 0.2271, -0.0165,  0.1382,  0.0907, -0.0504],
         [ 0.3176,  0.1364,  0.0548,  0.2315, -0.1117]]],
       grad_fn=<PermuteBackward>)

My out bacward:
tensor([[[-0.2995, -0.2097,  0.5120, -0.3699, -0.2247],
         [ 0.3512, -0.8475,  0.1094,  0.9039, -0.3021],
         [ 0.1210,  0.1893,  0.2093, -0.2455, -0.1848]],

        [[-0.2437,  0.2922,  0.0279, -0.4758,  0.1251],
         [-0.0109, -0.3373,  0.3782,  0.1597, -0.3190],
         [ 0.2642, -0.3012,  0.2351,  0.2848, -0.2415]]],
       grad_fn=<FlipBackward>)


In [113]:
print("Pytorch final output: ")
print(out)

my_final_output = torch.cat((my_out_forward, my_out_backward), -1)
print()
print("my final output: ")
print(my_final_output)

Pytorch final output: 
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897, -0.2995, -0.2097,
           0.5120, -0.3699, -0.2247],
         [ 0.1449,  0.0654,  0.0732,  0.6874,  0.3344,  0.3512, -0.8475,
           0.1094,  0.9039, -0.3021],
         [ 0.2900,  0.1927,  0.0584, -0.1282, -0.3534,  0.1210,  0.1893,
           0.2093, -0.2455, -0.1848]],

        [[ 0.0427,  0.0461, -0.0034, -0.2824, -0.0789, -0.2437,  0.2922,
           0.0279, -0.4758,  0.1251],
         [ 0.2271, -0.0165,  0.1382,  0.0907, -0.0504, -0.0109, -0.3373,
           0.3782,  0.1597, -0.3190],
         [ 0.3176,  0.1364,  0.0548,  0.2315, -0.1117,  0.2642, -0.3012,
           0.2351,  0.2848, -0.2415]]], grad_fn=<TransposeBackward1>)

my final output: 
tensor([[[ 0.3606,  0.1887, -0.1284, -0.0933, -0.0897, -0.2995, -0.2097,
           0.5120, -0.3699, -0.2247],
         [ 0.1449,  0.0654,  0.0732,  0.6874,  0.3344,  0.3512, -0.8475,
           0.1094,  0.9039, -0.3021],
         [ 0.2900,  0.1927,  0.0584,